<a href="https://colab.research.google.com/github/ayushi15092002/mental-health-workload/blob/main/foof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
clean_folder = "/content/drive/My Drive/drdo/cleanData"
results_foldername = "/content/drive/My Drive/drdo/spectral_analysis"

In [ ]:
pip install mne --upgrade
pip install fooof
pip install signal-processing
pip install basic

In [ ]:
import os, mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fooof import FOOOF
from fooof.bands import Bands
from fooof.analysis import get_band_peak_fm, get_band_peak_fg
from fooof.plts.periodic import plot_peak_fits, plot_peak_params
from fooof.plts.aperiodic import plot_aperiodic_params, plot_aperiodic_fits

In [ ]:
read_datasetA(clean_data_dir)

In [ ]:
# Define frequency bands of interest
bands = Bands({'delta' : [1, 4],
               'theta' : [4, 7.5],
               'alpha' : [7.5, 13],
               'beta' : [13, 30]})
clean_data_dir = "/content/drive/My Drive/drdo/cleanData"

subject_names = ['VP001_nback1','VP001_nback2','VP001_nback3',
                 'VP002_nback1','VP002_nback2','VP002_nback3',
                 'VP003_nback1','VP003_nback2','VP003_nback3',
                 'VP004_nback1','VP004_nback2','VP004_nback3',
                 'VP005_nback1','VP005_nback2','VP005_nback3',
                 'VP006_nback1','VP006_nback2','VP006_nback3',
                 'VP007_nback1','VP007_nback2','VP007_nback3',
                 'VP008_nback1','VP008_nback2','VP008_nback3',
                 'VP009_nback1','VP009_nback2','VP009_nback3',]

root, dirs, files = next(os.walk(clean_data_dir))

brain_regions = {'Frontal':['AFF5','F1','FP1','FC5','FC1','AFF6','F2','FP2','FC6','FC2','AFz'],
                 'Parietal':['CP5','CP1','P7','P3','O1','CP6','CP2','P8','P4','O2','Pz','POz']}

# shape: [region][subject]
deltas = [[None]*len(dirs)*3]*len(brain_regions.keys())
thetas = [[None]*len(dirs)*3]*len(brain_regions.keys())
alphas = [[None]*len(dirs)*3]*len(brain_regions.keys())
betas = [[None]*len(dirs)*3]*len(brain_regions.keys())
aps = [[None]*len(dirs)*3]*len(brain_regions.keys())
exp_condition  = ""
exp_folder = r""

freq_range = [1,30]
# note: probably have to re-preprocess the data with higher high-cutoff frequency than the used freq in here
# to avoid that part of the higher frequency spectra gets cut off which affects model fitting



NameError: ignored

In [ ]:
def read_datasetA(subjectPath):
    
    try:
        root, dirs, files = next(os.walk(subjectPath))

        i=0
        
        for folder_name in dirs:
          path_file_fif1 = subjectPath + '/' + folder_name + '/nback1.fif'
          # data_vis_A(i,path_file_fif1, folder_name,'nback1')
          file_path = path_file_fif1
          subject_name = folder_name + "_" + "nback1"
          epochs = mne.read_epochs(fname=file_path,verbose=False)
          # Calculate Welch's power spectrum density (FFT) -> (epochs, channels, freq bins) shape
          [psds,freqs] = calculate_psd(epochs,subject_name,fminmax=freq_range,window='hamming',
                                 window_duration=2,window_overlap=0.5,plot=False)
          # Average all epochs and channels together -> (freq bins,) shape
          psds_allch = psds.mean(axis=(0,1))

          psds = psds.mean(axis=(0))
          df_psds_ch = array_to_df(subject_name,epochs,psds).reset_index().drop(columns='Subject')

          df_psds_regions = df_channels_to_regions(df_psds_ch,brain_regions).reset_index().drop(columns='Subject')

          for region in df_psds_regions.columns:
            if i == 0:
              globals()["df_fooof_"+region] = pd.DataFrame(index=subject_names,columns=['Exponent','Offset','Peaks_CF','Peaks_PW','Peaks_BW','R_2','Error'])

            psds_temp = df_psds_regions[region].to_numpy()
            # Initialize power spectrum model objects and fit the power spectra
            # note: aperiodic_mode should be 'knee' if the frequency range is > 40 Hz
            max_n_peaks = float("inf")
            fm = FOOOF(peak_width_limits=[1,12],max_n_peaks=max_n_peaks,min_peak_height=0.225,
                peak_threshold=2.0,aperiodic_mode='fixed',verbose=True)
            fm.fit(freqs,psds_temp,freq_range)
            # Plot the fitted model with power spectra 
            fig = plt.figure()
            ax = fig.add_axes([0,0,1,1])
            ax.set_title('PSD, {}'.format(subject_name))
            if fm.get_params('peak_params').shape==(3,) or max_n_peaks == 0:
              fm.plot(ax=ax)
            else:
              fm.plot(plot_peaks='shade', peak_kwargs={'color' : 'green'},ax=ax)

            # Add model parameters to dataframe
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Exponent'] = fm.get_params('aperiodic_params','exponent')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Offset'] = fm.get_params('aperiodic_params','offset')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Peaks_CF'] = fm.get_params('peak_params', 'CF')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Peaks_PW'] = fm.get_params('peak_params', 'PW')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Peaks_BW'] = fm.get_params('peak_params', 'BW')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'R_2'] = fm.get_params('r_squared')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Error'] = fm.get_params('error')

# ///////////////////////////////////////////////////////////////////////////////////////
          path_file_fif2 = subjectPath + '/' + folder_name + '/nback2.fif'
          # data_vis_A(i,path_file_fif1, folder_name,'nback2')

          file_path = path_file_fif1
          subject_name = folder_name + "_" + "nback2"
          epochs = mne.read_epochs(fname=file_path,verbose=False)
          # Calculate Welch's power spectrum density (FFT) -> (epochs, channels, freq bins) shape
          [psds,freqs] = calculate_psd(epochs,subject_name,fminmax=freq_range,window='hamming',
                                 window_duration=2,window_overlap=0.5,plot=False)
          # Average all epochs and channels together -> (freq bins,) shape
          psds_allch = psds.mean(axis=(0,1))

          psds = psds.mean(axis=(0))
          df_psds_ch = array_to_df(subject_name,epochs,psds).reset_index().drop(columns='Subject')

          df_psds_regions = df_channels_to_regions(df_psds_ch,brain_regions).reset_index().drop(columns='Subject')

          for region in df_psds_regions.columns:
            if i == 0:
              globals()["df_fooof_"+region] = pd.DataFrame(index=subject_names,columns=['Exponent','Offset','Peaks_CF','Peaks_PW','Peaks_BW','R_2','Error'])

            psds_temp = df_psds_regions[region].to_numpy()
            # Initialize power spectrum model objects and fit the power spectra
            # note: aperiodic_mode should be 'knee' if the frequency range is > 40 Hz
            max_n_peaks = float("inf")
            fm = FOOOF(peak_width_limits=[1,12],max_n_peaks=max_n_peaks,min_peak_height=0.225,
                peak_threshold=2.0,aperiodic_mode='fixed',verbose=True)
            fm.fit(freqs,psds_temp,freq_range)
            # Plot the fitted model with power spectra 
            fig = plt.figure()
            ax = fig.add_axes([0,0,1,1])
            ax.set_title('PSD, {}'.format(subject_name))
            if fm.get_params('peak_params').shape==(3,) or max_n_peaks == 0:
              fm.plot(ax=ax)
            else:
              fm.plot(plot_peaks='shade', peak_kwargs={'color' : 'green'},ax=ax)

            # Add model parameters to dataframe
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Exponent'] = fm.get_params('aperiodic_params','exponent')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Offset'] = fm.get_params('aperiodic_params','offset')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Peaks_CF'] = fm.get_params('peak_params', 'CF')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Peaks_PW'] = fm.get_params('peak_params', 'PW')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Peaks_BW'] = fm.get_params('peak_params', 'BW')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'R_2'] = fm.get_params('r_squared')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Error'] = fm.get_params('error')

# ///////////////////////////////////////////////////////////////////////////////////////

          path_file_fif3 = subjectPath + '/' + folder_name + '/nback3.fif'
          # data_vis_A(i,path_file_fif1, folder_name,'nback3')

          file_path = path_file_fif1
          subject_name = folder_name + "_" + "nback3"
          epochs = mne.read_epochs(fname=file_path,verbose=False)
          # Calculate Welch's power spectrum density (FFT) -> (epochs, channels, freq bins) shape
          [psds,freqs] = calculate_psd(epochs,subject_name,fminmax=freq_range,window='hamming',
                                 window_duration=2,window_overlap=0.5,plot=False)
          # Average all epochs and channels together -> (freq bins,) shape
          psds_allch = psds.mean(axis=(0,1))

          psds = psds.mean(axis=(0))
          df_psds_ch = array_to_df(subject_name,epochs,psds).reset_index().drop(columns='Subject')

          df_psds_regions = df_channels_to_regions(df_psds_ch,brain_regions).reset_index().drop(columns='Subject')

          for region in df_psds_regions.columns:
            if i == 0:
              globals()["df_fooof_"+region] = pd.DataFrame(index=subject_names,columns=['Exponent','Offset','Peaks_CF','Peaks_PW','Peaks_BW','R_2','Error'])

            psds_temp = df_psds_regions[region].to_numpy()
            # Initialize power spectrum model objects and fit the power spectra
            # note: aperiodic_mode should be 'knee' if the frequency range is > 40 Hz
            max_n_peaks = float("inf")
            fm = FOOOF(peak_width_limits=[1,12],max_n_peaks=max_n_peaks,min_peak_height=0.225,
                peak_threshold=2.0,aperiodic_mode='fixed',verbose=True)
            fm.fit(freqs,psds_temp,freq_range)
            # Plot the fitted model with power spectra 
            fig = plt.figure()
            ax = fig.add_axes([0,0,1,1])
            ax.set_title('PSD, {}'.format(subject_name))
            if fm.get_params('peak_params').shape==(3,) or max_n_peaks == 0:
              fm.plot(ax=ax)
            else:
              fm.plot(plot_peaks='shade', peak_kwargs={'color' : 'green'},ax=ax)

            # Add model parameters to dataframe
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Exponent'] = fm.get_params('aperiodic_params','exponent')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Offset'] = fm.get_params('aperiodic_params','offset')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Peaks_CF'] = fm.get_params('peak_params', 'CF')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Peaks_PW'] = fm.get_params('peak_params', 'PW')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Peaks_BW'] = fm.get_params('peak_params', 'BW')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'R_2'] = fm.get_params('r_squared')
            globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Error'] = fm.get_params('error')

          i = i+1
        # Calculate alpha asymmetry for three channel pairs

        # print("df_psd_Alpha >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
        # print(df_psd_Alpha)
        # df_faa['Pre-frontal'] = calculate_asymmetry_ch(df_psd_Alpha,'FP1','FP2')
        # df_faa['Mid-frontal'] = calculate_asymmetry_ch(df_psd_Alpha,'F1','F2')
        # df_faa['Lateral-frontal'] = calculate_asymmetry_ch(df_psd_Alpha,'FC5','FC6')
        # Export FOOOF data for all regions
        for region in df_psds_regions.columns:
          globals()["df_fooof_"+region].to_excel(r"{}/FOOOF/{}_{}_fooof.xlsx".format(results_foldername,exp_folder,region))
          display(globals()["df_fooof_"+region])


        
        # Pre-create results folders for spectral analysis data
        # create_results_folders(exp_folder=exp_folder,results_foldername=results_foldername)

        # Export asymmetry data
        # df_faa.to_excel(r"{}/Absolute PSD/{}_frontal_asymmetry.xlsx".format(results_foldername,exp_condition))

        # Export power spectra data for each band
        # for band in b_names:
        #   export_psd_results(globals()["df_psd_"+band],globals()["df_rel_psd_"+band],exp_folder,exp_condition,band,brain_regions,results_foldername=results_foldername)
            
    except StopIteration:
        pass
        print("Error ocurred:")
        print("Directory with dataset does not found!")
        print("Program will be terminated")
        exit(1)




In [ ]:
def data_vis_A(i, file_path, subject_name, sesssion_name):
  subject_name = subject_name + "_" + sesssion_name
  epochs = mne.read_epochs(fname=file_path,verbose=False)
  # Calculate Welch's power spectrum density (FFT) -> (epochs, channels, freq bins) shape
  [psds,freqs] = calculate_psd(epochs,subject_name,fminmax=freq_range,window='hamming',
                                 window_duration=2,window_overlap=0.5,plot=False)
  # Average all epochs and channels together -> (freq bins,) shape
  psds_allch = psds.mean(axis=(0,1))

  psds = psds.mean(axis=(0))
  df_psds_ch = array_to_df(subject_name,epochs,psds).reset_index().drop(columns='Subject')

  df_psds_regions = df_channels_to_regions(df_psds_ch,brain_regions).reset_index().drop(columns='Subject')

  for region in df_psds_regions.columns:
    if i == 0:
      globals()["df_fooof_"+region] = pd.DataFrame(index=subject_names,columns=['Exponent','Offset','Peaks_CF','Peaks_PW','Peaks_BW','R_2','Error'])

    psds_temp = df_psds_regions[region].to_numpy()
    # Initialize power spectrum model objects and fit the power spectra
    # note: aperiodic_mode should be 'knee' if the frequency range is > 40 Hz
    max_n_peaks = float("inf")
    fm = FOOOF(peak_width_limits=[1,12],max_n_peaks=max_n_peaks,min_peak_height=0.225,
                peak_threshold=2.0,aperiodic_mode='fixed',verbose=True)
    fm.fit(freqs,psds_temp,freq_range)
    # Plot the fitted model with power spectra 
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.set_title('PSD, {}'.format(subject_name))
    if fm.get_params('peak_params').shape==(3,) or max_n_peaks == 0:
      fm.plot(ax=ax)
    else:
      fm.plot(plot_peaks='shade', peak_kwargs={'color' : 'green'},ax=ax)

    # Add model parameters to dataframe
    globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Exponent'] = fm.get_params('aperiodic_params','exponent')
    globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Offset'] = fm.get_params('aperiodic_params','offset')
    globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Peaks_CF'] = fm.get_params('peak_params', 'CF')
    globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Peaks_PW'] = fm.get_params('peak_params', 'PW')
    globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Peaks_BW'] = fm.get_params('peak_params', 'BW')
    globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'R_2'] = fm.get_params('r_squared')
    globals()["df_fooof_"+region].loc[globals()["df_fooof_"+region].index[i],'Error'] = fm.get_params('error')



In [ ]:
# ========== Packages ==========
import mne
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# ========== Functions ==========
def calculate_psd(epochs,subjectname,fminmax=[1,100],window='hamming',window_duration=2,window_overlap=0.5,plot=True):
    """
    Calculate power spectrum density with FFT/Welch's method and plot the result.
    Parameters
    ----------
    epochs: Epochs-type (MNE-Python) EEG file
    subjectname: A string for subject's name
    fminmax (optional): The minimum and maximum frequency range for estimating Welch's PSD
    window (optional): The window type for estimating Welch's PSD
    window_duration (optional): An integer for the length of that window
    window_overlap (optional): A float for the percentage of overlap between the windows
    Returns
    -------
    psds: An array for power spectrum density values
    freqs: An array for the corresponding frequencies
    """
    # Calculate PSD with Welch's method
    window_size = int(epochs.info['sfreq']*window_duration)
    psds, freqs = mne.time_frequency.psd_welch(epochs,n_fft=window_size,verbose=False,
                                               n_overlap=int(window_size*window_overlap),
                                               fmin=fminmax[0],fmax=fminmax[1],window=window)

    # Unit conversion from V^2/Hz to uV^2/Hz
    psds = psds*1e12

    if plot == True:
        # Plot average PSD for all epochs and channels (only for plot)
        psds_mean_all = psds.mean(axis=(0, 1))

        sns.set_style("darkgrid",{'font.family': ['sans-serif']})
        plt.figure()
        plt.plot(freqs,psds_mean_all)
        plt.fill_between(freqs,psds_mean_all)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('PSD (uV\u00b2/Hz)')
        plt.title("PSD ({})".format(subjectname))
        plt.xlim(1,40)
        plt.ylim(0,1.1*max(psds_mean_all))

    return [psds,freqs]

def siqnal_quality_check(psds,freqs,band,b_name,subjectname,epochs):
    """
    Plot topographically PSD values and calculate median absolute deviation
    for the first and second half of the signal for signal reliability control.
    Parameters
    ----------
    psds: An array for power spectrum density values
    freqs: An array for corresponding frequencies
    band: A list of lower and higher frequency for the frequency band in interest
    b_name: A string for frequency band in interest
    subjectname: A string for subject's name
    epochs: Epochs-type (MNE-Python) EEG file
    Returns
    -------
    psd_max_mad_error: A float for max median absolute deviation for PSD values
    """

    # Divide the epochs into two
    idx_mid_epoch = round(len(psds[:,0,0])/2)
    psds_p1 = psds[:idx_mid_epoch,:,:]
    psds_p2 = psds[idx_mid_epoch:,:,:]

    # Define the bandpower indices
    low, high = band
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Get only the bandpower values that we need for both parts
    psd_band_ch_p1 = psds_p1[:,:,idx_band]
    psd_band_mean_epochs_p1 = psd_band_ch_p1.mean(axis=(2))
    psd_band_ch_p2 = psds_p2[:,:,idx_band]
    psd_band_mean_epochs_p2 = psd_band_ch_p2.mean(axis=(2))

    # Find Median Absolute Deviation along all the epochs for each channel
    psd_band_mean_epochs = np.concatenate((psd_band_mean_epochs_p1, psd_band_mean_epochs_p2))
    psd_median = np.median(psd_band_mean_epochs,axis=0)
    psd_mad = stats.median_abs_deviation(psd_band_mean_epochs,axis=0)

    # For each epoch in all channels, calculate the Z-score (MAD error) using median and MAD
    psd_mad_error_epochs_per_ch_p1 = np.zeros(shape=psd_band_mean_epochs_p1.shape)
    psd_mad_error_epochs_per_ch_p2 = np.zeros(shape=psd_band_mean_epochs_p2.shape)
    for i in range(len(psds[0,:,0])):
        psd_mad_error_epochs_per_ch_p1[:,i] = abs(psd_band_mean_epochs_p1[:,i] - psd_median[i])/psd_mad[i]
        psd_mad_error_epochs_per_ch_p2[:,i] = abs(psd_band_mean_epochs_p2[:,i] - psd_median[i])/psd_mad[i]

    # Average the Z-scores (MAD errors) for all epochs for the two parts -> get score for 19 channels
    psd_mad_error_per_ch_p1 = np.mean(psd_mad_error_epochs_per_ch_p1,axis=0)
    psd_mad_error_per_ch_p2 = np.mean(psd_mad_error_epochs_per_ch_p2,axis=0)

    # Find the average Z-score (MAD error) for the whole scalp (all channels together) for the two parts
    psd_mad_error_avg = [np.round(np.mean(psd_mad_error_per_ch_p1),3),
                         np.round(np.mean(psd_mad_error_per_ch_p2),3)]
    
    # Get the maximum of the two (for outputting it later)
    psd_max_mad_error = np.max(psd_mad_error_avg)

    # Find the average PSD values for each channel for both parts
    psds_all_channels_p1 = psds_p1.mean(axis=(0))
    psd_band_ch_p1 = psds_all_channels_p1[:,idx_band]
    psd_band_mean_ch_p1 = psd_band_ch_p1.mean(axis=(1))
    psds_all_channels_p2 = psds_p2.mean(axis=(0))
    psd_band_ch_p2 = psds_all_channels_p2[:,idx_band]
    psd_band_mean_ch_p2 = psd_band_ch_p2.mean(axis=(1))

    
    # Visually display the bandpower for both parts for visual inspection
    vmin = min([min(psd_band_mean_ch_p1),min(psd_band_mean_ch_p2)])
    vmax = max([max(psd_band_mean_ch_p1),max(psd_band_mean_ch_p2)])

    if psd_max_mad_error >= 2:
        sns.set_style("white",{'font.family': ['sans-serif']})
        fig,(ax1,ax2) = plt.subplots(ncols=2)
        fig.suptitle("MAD >= 2 ! Quality control for {} ({})".format(b_name,subjectname),y=1.1,x=0.575)
        im,cm = mne.viz.plot_topomap(psd_band_mean_ch_p1,epochs.info,axes=ax1,vlim=[vmin,vmax],show=False)
        im,cm = mne.viz.plot_topomap(psd_band_mean_ch_p2,epochs.info,axes=ax2,vlim=[vmin,vmax],show=False)
        ax1.set_title("Epochs 0-{}\nAvg MAD error = {}".format(idx_mid_epoch-1,psd_mad_error_avg[0]))
        ax2.set_title("Epochs {}-{}\nAvg MAD error = {}".format(idx_mid_epoch,len(psds[:,0,0]),psd_mad_error_avg[1]))
        ax_x_start = 0.95
        ax_x_width = 0.04
        ax_y_start = 0.1
        ax_y_height = 0.9
        cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
        clb = fig.colorbar(im, cax=cbar_ax)
        clb.ax.set_ylabel('uV\u00b2/Hz'); 

    return psd_max_mad_error

def bandpower_per_channel(psds,freqs,band,b_name,subjectname,epochs,ln_normalization=False):
    """
    Find frequency band power in interest for all the channels.
    Parameters
    ----------
    psds: An array for power spectrum density values
    freqs: An array for corresponding frequencies
    band: A list of lower and higher frequency for the frequency band in interest
    b_name: A string for frequency band in interest
    subjectname: A string for subject's name
    epochs: Epochs-type (MNE-Python) EEG file
    Returns
    -------
    psd_band_mean_ch: An array for a frequency band power values for all the channels.
    """
    # Calculate the MAD error (z-score) of the bandpower to be sure of the quality
    psd_max_mad_error = siqnal_quality_check(psds,freqs,band,b_name,subjectname,epochs)
    
    low, high = band
    psds_all_channels = psds.mean(axis=(0))
    idx_band = np.logical_and(freqs >= low, freqs <= high)
    psd_band_ch = psds_all_channels[:,idx_band]
    psd_band_mean_ch = psd_band_ch.mean(axis=(1))

    #print(band,'before norm',psd_band_mean_ch)
    if ln_normalization == True:
        psd_band_mean_ch = np.log(psd_band_mean_ch)

    #print(band,'after norm',psd_band_mean_ch)
    # # If the error is larger than 2, print it as a result (visual inspection)
    # if psd_max_mad_error < 2:
    #     print(subjectname,b_name,"MAD error is OK:",psd_max_mad_error)
    # else:
    #     print(subjectname,b_name,"MAD error is NOT OK:",psd_max_mad_error)

    return psd_band_mean_ch

def calculate_asymmetry_ch(df_psd_band,left_ch,right_ch):
    """
    Calculate asymmetry between brain hemispheres.
    Parameters
    ----------
    df_psd_band: A dataframe with PSD values (for each region/channel) per subject for one band
    left_ch: A string for the left channel (or region)
    right_ch: A string for the right channel (or region)
    Returns
    -------
    df_asymmetry: A dataframe for calculated asymmetry for all the subjects
    """

    print("df_psd_band[left_ch] >>>>>>>>>>>>>>>>>>>>>>>")
    # print(df_psd_band[left_ch])
    df_asymmetry = (df_psd_band[left_ch] - df_psd_band[right_ch])/(df_psd_band[left_ch] + df_psd_band[right_ch])*100
    return df_asymmetry

In [ ]:
# ========== Packages ==========
import os
import pandas as pd

# ========== Functions ==========
def read_files(dir_inprogress,filetype,exclude_subjects=None,verbose=True):
    """
    Get all the (EEG) file directories and subject names.
    Parameters
    ----------
    dir_inprogress: A string with directory to look for files
    filetype: A string with the ending of the files we are looking for (e.g. '.bdf')
    Returns
    -------
    file_dirs: A list of strings with file directories for all the (EEG) files
    subject_names: A list of strings with all the corresponding subject names
    """
    file_dirs = []
    subject_names = []
    for file in os.listdir(dir_inprogress):
        if file.endswith(filetype):
            file_dirs.append(os.path.join(dir_inprogress, file))
            subject_names.append(os.path.join(file).removesuffix(filetype))

    for excl_sub in exclude_subjects:
        for i in range(len(subject_names)):
            if excl_sub in subject_names[i]:
                if verbose == True:
                    print('EXCLUDED SUBJECT: ',excl_sub,'in',subject_names[i],'at',file_dirs[i])
                del subject_names[i]
                del file_dirs[i]
                break

    if verbose == True:
        print("Files in folder read in:",len(file_dirs))

    return [file_dirs, subject_names]

def array_to_df(subjectname,epochs,array_channels):
    """
    Convert channel-based array to Pandas dataframe with channels' and subjects' names.
    Parameters
    ----------
    subjectname: A string for subject's name
    epochs: Epochs-type (MNE-Python) EEG file
    array_channels: An array with values for each channel
    Returns
    -------
    df_channels: A dataframe with values for each channel
    """
    df_channels = pd.DataFrame(array_channels).T
    df_channels.columns = epochs.info.ch_names
    df_channels['Subject'] = subjectname
    df_channels.set_index('Subject', inplace=True)

    return df_channels

def df_channels_to_regions(df_psd_band,brain_regions):
    """
    Average channels together based on the defined brain regions.
    Parameters
    ----------
    df_psd_band: A dataframe with PSD values for each channel per subject
    brain_regions: A dictionary of brain regions and EEG channels which they contain
    Returns
    -------
    df_psd_reg_band: A dataframe with PSD values for each brain region per subject
    """
    df_psd_reg_band = pd.DataFrame()
    # print("brain_regions >>>>>>>>>>>>>>>>>>>>>>>>>>>")
    # print(brain_regions)
    for reg_name in brain_regions:
      # print("df_psd_band[brain_regions[reg_name]] >>>>>>>>>>>>>>>>>>>>")
      # print(df_psd_band[brain_regions[reg_name]])
      df_temp = df_psd_band[brain_regions[reg_name]].copy().mean(axis=1)
      df_psd_reg_band = pd.concat([df_psd_reg_band,df_temp],axis=1)
        
    df_psd_reg_band.columns = brain_regions.keys()
    df_psd_reg_band.index.name = 'Subject'

    return df_psd_reg_band

def read_excel_psd(exp_folder,psd_folder,verbose=True):
    """
    Get all PSD file directories and corresponding bands and experiment conditions.
    Parameters
    ----------
    exp_folder: A string with a relative directory to experiment folder (e.g. 'Eyes Closed\Baseline')
    psd_folder: A string with a relative directory to the results folder (e.g. 'Results\PSD\regions')
    Returns
    -------
    dir_inprogress: A string with directory to look for files
    b_names: A list of strings for frequency bands of the files
    condition: A list of strings for experiment conditions of the files
    """
    dir_inprogress = os.path.join(psd_folder,exp_folder)
    _, b_names = read_files(dir_inprogress,".xlsx",verbose=verbose)

    condition = [None]*len(b_names)
    for i in range(len(b_names)):
        condition[i] = b_names[i].split("_psd_", 1)
    
    return [dir_inprogress,b_names,condition]

def create_results_folders(exp_folder,results_foldername="Results"):
    """
    Dummy way to try to pre-create folders for PSD results before exporting them
    Parameters
    ----------
    exp_folder: A string with a relative directory to experiment folder (e.g. 'Eyes Closed\Baseline')
    """
    try:
        os.makedirs(os.path.join(r"{}/Absolute PSD/channels".format(results_foldername),exp_folder))
    except FileExistsError:
        pass
    try:
        os.makedirs(os.path.join(r"{}/Absolute PSD/regions".format(results_foldername),exp_folder))
    except FileExistsError:
        pass
    try:
        os.makedirs(os.path.join(r"{}/Relative PSD/channels".format(results_foldername),exp_folder))
    except FileExistsError:
        pass
    try:
        os.makedirs(os.path.join(r"{}/Relative PSD/regions".format(results_foldername),exp_folder))
    except FileExistsError:
        pass
    try:
        os.makedirs(os.path.join(r"{}/FOOOF".format(results_foldername),exp_folder))
    except FileExistsError:
        pass

def export_psd_results(df_psd_band,df_rel_psd_band,exp_folder,exp_condition,band,brain_regions,results_foldername='Results'):
    """
    Export PSD results (for channels & regions) as Excel files
    Parameters
    ----------
    df_psd_band: A dataframe with PSD values for each channel per subject
    df_rel_psd_band: A dataframe with relative PSD values for each channel per subject
    exp_folder: A string with a relative directory to experiment folder (e.g. 'Eyes Closed\Baseline')
    exp_condition: A string for experiment short code (e.g. 'EC_00')
    band: The frequency band of the PSD values (e.g. 'Alpha')
    brain_regions: A dictionary of brain regions and EEG channels which they contain
    """
    # Save the PSD values for each channel for each band in Excel format
    df_psd_band.to_excel(r"{}/Absolute PSD/channels/{}/{}_psd_{}.xlsx".format(results_foldername,exp_folder,exp_condition,band))
    df_rel_psd_band.to_excel(r"{}/Relative PSD/channels/{}/{}_rel_psd_{}.xlsx".format(results_foldername,exp_folder,exp_condition,band))

    # Find regional band powers and save them to Excel as well
    df_psd_band_reg = df_channels_to_regions(df_psd_band,brain_regions)
    df_psd_band_reg.to_excel(r"{}/Absolute PSD/regions/{}/{}_psd_{}.xlsx".format(results_foldername,exp_folder,exp_condition,band))
    df_rel_psd_band_reg = df_channels_to_regions(df_rel_psd_band,brain_regions)
    df_rel_psd_band_reg.to_excel(r"{}/Relative PSD/regions/{}/{}_rel_psd_{}.xlsx".format(results_foldername,exp_folder,exp_condition,band))